### Feature Extraction
- pkl을 로드해서 사용하는 부분 next_batch에 추가하기 (given이 true이면 하고 아니면 원래대로 하는 것으로)
- 한 노래 돌때 피쳐 모두 뽑아서 각각 저장하기

In [2]:
import os
import numpy as np
import pandas as pd
import features
from scipy import stats
import librosa
from tqdm import *
import pickle

In [63]:
def test(feature_name, split_method, is_save=False, HOP_LENGTH=None, N_FFT=None, N_CHROMA=None, N_BANDS=None):
    if is_save:
        return save_output_as_pkl(
            output = get_tid_feature_pairs(
                    get_track_ids(
                        split_df(
                            build_metadata_df('dataset/track_metadata.csv')
                            , split_method
                        )
                    )
                    , feature_name
                    , HOP_LENGTH
                    , N_FFT
                    , N_CHROMA
                    , N_BANDS
                )
            , filename='{}_{}.pkl'.format(feature_name, split_method)
        )
    else:
        return get_tid_feature_pairs(
                    get_track_ids(
                        split_df(
                            build_metadata_df('dataset/track_metadata.csv')
                            , split_method
                        )
                    )
                    , feature_name
                    , HOP_LENGTH
                    , N_FFT
                    , N_CHROMA
                    , N_BANDS
                )

def build_metadata_df(filepath):
    return pd.read_csv(filepath)

def split_df(metadata_df, split_method):
    return metadata_df[metadata_df['set_split'] == split_method]

def get_track_ids(metadata_df):
    return metadata_df['track_id'].values

def save_output_as_pkl(output, filename):
    with open(filename, 'wb') as f:
        pickle.dump(output, f)

test('chroma_stft', 'training', is_save=True, HOP_LENGTH=512*8, N_FFT=2048, N_CHROMA=20)
test('chroma_stft', 'validation', is_save=True, HOP_LENGTH=512*8, N_FFT=2048, N_CHROMA=20)
#print(test('rmse', 'training', HOP_LENGTH=512*8, N_FFT=2048)[0][1].shape)
#print(test('spectral_contrast', 'training', HOP_LENGTH=512*8, N_BANDS=6)[0][1].shape)
#test('rmse', 'training')
#test('rmse', 'validation')
#test('spectral_contrast', 'training')
#test('spectral_contrast', 'validation')

# TODO : dictionary 로 track_id : array로 받아서 나중에 array로 바꾸는게 낫지 않을까? 현재는 일단 그냥 함.
# TODO : get_feature_by_tid로 바꾸기

/Users/dongmin.shin/anaconda3/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


139491: ValueError('song length is shorter than threshold',)
139520: FileNotFoundError(2, 'No such file or directory')
139521: FileNotFoundError(2, 'No such file or directory')
139522: FileNotFoundError(2, 'No such file or directory')
139523: FileNotFoundError(2, 'No such file or directory')
139524: FileNotFoundError(2, 'No such file or directory')
139532: FileNotFoundError(2, 'No such file or directory')
139534: FileNotFoundError(2, 'No such file or directory')
139536: FileNotFoundError(2, 'No such file or directory')
139537: FileNotFoundError(2, 'No such file or directory')
139637: FileNotFoundError(2, 'No such file or directory')
139638: FileNotFoundError(2, 'No such file or directory')
139663: FileNotFoundError(2, 'No such file or directory')
139681: FileNotFoundError(2, 'No such file or directory')
139687: FileNotFoundError(2, 'No such file or directory')
139688: FileNotFoundError(2, 'No such file or directory')
139769: FileNotFoundError(2, 'No such file or directory')
139770: Fil

140583: FileNotFoundError(2, 'No such file or directory')
140935: FileNotFoundError(2, 'No such file or directory')
140939: FileNotFoundError(2, 'No such file or directory')
141735: FileNotFoundError(2, 'No such file or directory')
142094: FileNotFoundError(2, 'No such file or directory')
142096: FileNotFoundError(2, 'No such file or directory')
142097: FileNotFoundError(2, 'No such file or directory')
142098: FileNotFoundError(2, 'No such file or directory')
142433: FileNotFoundError(2, 'No such file or directory')
142563: FileNotFoundError(2, 'No such file or directory')
142641: FileNotFoundError(2, 'No such file or directory')
142947: FileNotFoundError(2, 'No such file or directory')
142948: FileNotFoundError(2, 'No such file or directory')
142949: FileNotFoundError(2, 'No such file or directory')
142950: FileNotFoundError(2, 'No such file or directory')
143186: FileNotFoundError(2, 'No such file or directory')
143239: FileNotFoundError(2, 'No such file or directory')
143295: FileNo

In [59]:
def get_tid_feature_pairs(tids, feature_name, HOP_LENGTH, N_FFT, N_CHROMA, N_BANDS):
    tid_feature_pairs = list()
    
    for tid in tqdm_notebook(tids):
        try:
            x, sr = load_audio_as_librosa_format(
                get_audio_path('music/music_training', tid)
                , DURATION=29.0
            )
            
            tid_feature_pairs.append((
                tid
                , np.array( 
                     compute_feature(
                         tid,
                         cut_x(x.tolist(), THRESHOLD=1278900)
                         , feature_name
                         , HOP_LENGTH
                         , N_FFT
                         , N_CHROMA
                         , N_BANDS
                     )
                )
            ))
        except Exception as e:
            print('{}: {}'.format(tid, repr(e)))

    return tid_feature_pairs

def get_audio_path(audio_dir, track_id):
    tid_str = '{:06d}'.format(track_id)
    return os.path.join(audio_dir, tid_str[:3], tid_str + '.mp3')

def load_audio_as_librosa_format(filepath, DURATION):
    return librosa.load(filepath, sr=None, mono=True, duration=DURATION)  # kaiser_fast

def cut_x(x, THRESHOLD):
    validate_x(x, THRESHOLD)
    return np.array(x[:THRESHOLD])

def validate_x(x, THRESHOLD):
    try:
        assert len(x) >= THRESHOLD
    except:
        raise ValueError('song length is shorter than threshold')
        
def compute_feature(tid, cutted_x, feature_name, HOP_LENGTH, N_FFT, N_CHROMA, N_BANDS):
    if feature_name == 'chroma_stft':
        feature = compute_chroma_stft(
            compute_stft(cutted_x, N_FFT, HOP_LENGTH)
            , N_CHROMA
        ).tolist()

    if feature_name == 'rmse':
        feature = compute_rmse(
            compute_stft(cutted_x, N_FFT, HOP_LENGTH)
        ).tolist()

    if feature_name == 'spectral_contrast':
        feature = compute_spectral_contrast(
            compute_stft(cutted_x, N_FFT, HOP_LENGTH)
            , N_BANDS
        ).tolist()
    return validate_feature(tid, feature)
    
def validate_feature(tid, feature):
    try:
        assert feature != 0
        return feature
    except:
        raise ValueError('tid {} has feature error'.format(tid))
    
def compute_stft(x, N_FFT, HOP_LENGTH):
    stft = np.abs(librosa.stft(x, n_fft=N_FFT, hop_length=HOP_LENGTH))
    return validate_stft(stft, x, N_FFT, HOP_LENGTH)

def validate_stft(stft, x, N_FFT, HOP_LENGTH):
    assert stft.shape[0] == 1 + N_FFT // 2
    assert np.ceil(len(x) / HOP_LENGTH) \
        <= stft.shape[1] \
        <= np.ceil(len(x) / HOP_LENGTH) + 1
    return stft

def compute_chroma_stft(stft, N_CHROMA):
    return librosa.feature.chroma_stft(S=stft ** 2, n_chroma=N_CHROMA) 
        
def compute_rmse(stft):
    return librosa.feature.rmse(S=stft)

def compute_spectral_contrast(stft, N_BANDS):
    print(N_BANDS)
    return librosa.feature.spectral_contrast(S=stft, n_bands=N_BANDS)